In [1]:
import os
import random
import gc
import time
import warnings

import numpy as np
import pandas as pd

from astropy.io import fits
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.wcs import WCS
from astropy.nddata.utils import Cutout2D
from astropy.table import Table
from reproject import reproject_exact
from scipy.stats import norm

from astropy.coordinates import SkyCoord
from astropy import units
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder

from photutils.background import MMMBackground
from photutils.psf import DAOGroup, IntegratedGaussianPRF, DAOPhotPSFPhotometry
from astropy.modeling.fitting import LevMarLSQFitter
from tqdm import tqdm
from photutils.aperture import CircularAperture, aperture_photometry

import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec


### Multiply the observational data maps by the correction factors and save it to disk

In [2]:
# Function to process the FITS files
def process_fits_files(input_filepath, output_filepath, corr_factor, max_hdu_idx=-1):
    # Get a list of all FITS files in the input directory
    # Open the FITS file
    with fits.open(input_filepath) as hdul:
        # apply the correction factor to all hdus, since errors propagate with the same correction factor
        for idx, hdu in enumerate(hdul):
            if max_hdu_idx != -1 and idx >= max_hdu_idx:
                break
            if hdu.data is not None:
                hdu.data *= corr_factor
        # Save the modified HDU to a new FITS file
        hdul.writeto(output_filepath, overwrite=True)

    print(f"Processed file: {input_filepath}\n")

In [3]:
input_filepaths = ["/mnt/d/SRHerschel500/data/raw/obs datamaps/mips_24_GO3_sci_10.fits", "/mnt/d/SRHerschel500/data/raw/obs datamaps/COSMOS-Nest_image_250_SMAP_v6.0.fits",
                    "/mnt/d/SRHerschel500/data/raw/obs datamaps/COSMOS-Nest_image_350_SMAP_v6.0.fits", "/mnt/d/SRHerschel500/data/raw/obs datamaps/COSMOS-Nest_image_500_SMAP_v6.0.fits",
                    "/mnt/d/SRHerschel500/data/raw/obs datamaps/SCS_450.fits"]

output_filepaths = ["/mnt/d/SRHerschel500/data/raw/obs datamaps/mips_24_GO3_sci_10_corr.fits",
                    "/mnt/d/SRHerschel500/data/raw/obs datamaps/COSMOS-Nest_image_250_SMAP_v6.0_corr.fits", 
                    "/mnt/d/SRHerschel500/data/raw/obs datamaps/COSMOS-Nest_image_350_SMAP_v6.0_corr.fits",
                    "/mnt/d/SRHerschel500/data/raw/obs datamaps/COSMOS-Nest_image_500_SMAP_v6.0_corr.fits",
                    "/mnt/d/SRHerschel500/data/raw/obs datamaps/SCS_450_corr.fits"]

corr_factor_l = [0.865e-3*1.369, 1, 1, 1, 1e-3]
# corr_factor_l = [0.141085553/1000 * 1.369, 1, 1, 1, 1e-3]
# corr_factor_l = [0.000865, 1, 1, 1, 1e-3]

for idx in range(len(input_filepaths)):
    process_fits_files(input_filepaths[idx], output_filepaths[idx], corr_factor_l[idx])

# Additionally, we have a seperate file for uncertainties of the 24 microns
process_fits_files("/mnt/d/SRHerschel500/data/raw/obs datamaps/mips_24_GO3_unc_10.fits", "/mnt/d/SRHerschel500/data/raw/obs datamaps/mips_24_GO3_unc_10_corr2.fits", corr_factor_l[0])

# Additionally, we have a seperate file for uncertainties of the 450 microns, but we can only correct until hdu idx 2 due to errors
process_fits_files("/mnt/d/SRHerschel500/data/raw/obs datamaps/SCS_450_rm_mf.fits", "/mnt/d/SRHerschel500/data/raw/obs datamaps/SCS_450_rm_mf_corr.fits", corr_factor_l[-1], max_hdu_idx=2)

Processed file: /mnt/d/SRHerschel500/data/raw/obs datamaps/mips_24_GO3_sci_10.fits

Processed file: /mnt/d/SRHerschel500/data/raw/obs datamaps/COSMOS-Nest_image_250_SMAP_v6.0.fits

Processed file: /mnt/d/SRHerschel500/data/raw/obs datamaps/COSMOS-Nest_image_350_SMAP_v6.0.fits

Processed file: /mnt/d/SRHerschel500/data/raw/obs datamaps/COSMOS-Nest_image_500_SMAP_v6.0.fits

Processed file: /mnt/d/SRHerschel500/data/raw/obs datamaps/SCS_450.fits

Processed file: /mnt/d/SRHerschel500/data/raw/obs datamaps/mips_24_GO3_unc_10.fits

Processed file: /mnt/d/SRHerschel500/data/raw/obs datamaps/SCS_450_rm_mf.fits



In [4]:
pixel_size = 1.2
beam_fwhm = 5.7
gaussian_fwhm_to_sigma = 0.42466
sigma_pix =  beam_fwhm * gaussian_fwhm_to_sigma / pixel_size

In [5]:
from astropy import units as u
beam_fwhm = 5.7*u.arcsec
fwhm_to_sigma = 1. / (8 * np.log(2))**0.5
beam_sigma = beam_fwhm * fwhm_to_sigma
omega_B = 2 * np.pi * beam_sigma**2
(1*u.MJy/u.sr).to(u.Jy/u.beam, equivalencies=u.beam_angular_area(omega_B))  

<Quantity 0.00086529 Jy / beam>